In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p import P2P
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist_vae", "../../data", batch_size=22)

2022-05-02 11:09:00.507577: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 11:09:00.871419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14252 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
model = P2P(weight_cpc=100.0, weight_align=0.0, skip_prob=0.1, beta=0.1)
# model.encoder(tf.zeros((input_shape)))
# model.encoder.summary()
#model.decoder(model.encoder(tf.ones((input_shape))))
#model.decoder.summary()
model(next(train_ds.as_numpy_iterator())[0])

2022-05-02 11:09:04.547126: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-02 11:09:05.600741: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(<tf.Tensor: shape=(), dtype=float32, numpy=0.20688848>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.005465192>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.012164119>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0031382102>)

In [6]:
iterator = train_ds.as_numpy_iterator()
for epoch in tqdm(range(200)):
    epoch_mse = 0
    epoch_kld = 0
    epoch_align = 0
    epoch_cpc = 0

    for x in train_ds.as_numpy_iterator():
        x = x[0]
        
        start_ix = 0
        cp_ix = -1
        cp_ix = x.shape[1] - 1

        mse, kld, cpc, align = model(x, start_ix, cp_ix)
        epoch_mse += mse
        epoch_kld += kld
        epoch_cpc += cpc
        epoch_align += align
    print("EPOCH", epoch)
    print("epoch mse", epoch_mse)
    print("epoch kld", epoch_kld)
    print("epoch cpc", epoch_cpc)
    print("epoch align", epoch_align)

  0%|                                                                                                                                                                          | 0/200 [03:05<?, ?it/s]


InvalidArgumentError: required broadcastable shapes [Op:AddV2]

In [ ]:
x = next(test_ds.as_numpy_iterator())
samples = model.p2p_generate(x[1], 20, 19)

In [ ]:
from ganime.visualization.videos import display_videos
display_videos(samples)